In [1]:
import torch
import timm
import einops
import tqdm
import cv2 as cv2
import numpy as np
import time
import os
import sys
from visualizer import Visualizer
import cv2
import time
from IPython.display import display, clear_output

In [25]:
def record_video(duration=5, video_name="output.mp4"):
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    # Define video codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_name, fourcc, 20.0, (640, 480))
    
    start_time = time.time()
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if ret:
            # Write the frame to the output video
            out.write(frame)
            
            # Display the frame in Jupyter Notebook
            clear_output(wait=True)
            display(cv2.imencode('.jpg', frame)[1].tostring())
        
        # Stop recording after the specified duration
        if time.time() - start_time > duration:
            break
    
    # Release the webcam and output video
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return video_name

In [3]:
def load_video(pth):
    video = cv2.VideoCapture(pth)
    video.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    video.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    video.set(cv2.CAP_PROP_FPS, 30)
    buffer = []
    while True:
        ret, frame = video.read()
        if not ret:
            break
        buffer.append(frame)
    return video, buffer

In [ ]:
# Run the function to record a 5-second video
vid = record_video(5, "my_video.mp4")
# or optionally set the vid varaible to the path of where you video is

In [4]:
cotracker = torch.hub.load("facebookresearch/co-tracker", "cotracker_w8")

Using cache found in /home/velocitatem/.cache/torch/hub/facebookresearch_co-tracker_main


In [5]:
video, buffer = load_video(vid)
video_tensor = torch.from_numpy(np.array(buffer)).permute(0, 3, 1, 2)[None].float()

In [11]:
if torch.cuda.is_available():
    cotracker = cotracker.cuda()
    video_tensor = video_torch.cuda()

In [12]:
tracks, visibility = cotracker(
    video_tensor,
    grid_size=4,
    grid_query_frame=0,
    backward_tracking=False
)

In [13]:
print(tracks.shape)
print(visibility.shape)

torch.Size([1, 145, 16, 2])
torch.Size([1, 145, 16])


In [14]:
np.save("tracks.npy", tracks.cpu().numpy())
np.save("visibility.npy", visibility.cpu().numpy())

In [15]:
from visualizer import Visualizer

In [18]:
vis = Visualizer(
    save_dir="./",
    grayscale=False,
    pad_value=100,
    fps=30,
    linewidth=2,
    show_first_frame=5,
    tracks_leave_trace=0
)

In [22]:
file = vis.visualize(
    video_tensor,
    tracks=tracks,
    visibility=visibility,
    filename="my_video_tracked",
    query_frame=0,
    )

Video saved to ./my_video_tracked_pred_track.mp4


In [24]:
from IPython.display import Video

Video("./my_video_tracked_pred_track.mp4")